# KMeans

## What is KMeans?

- KMeans is an unsupervised machine learning algorithm used for clustering data into **'K'** groups based on similarities.
- It **iteratively** refines the **clusters** by **adjusting the positions of cluster centroids** and reassigning data points to the nearest centroid until convergence.
- K-means is one of the most widely used clustering algorithms in machine learning.
- Unsupervised learning refers to learning without using pre-existing labels or categories (meaning the algorithm must discover patterns or groups within the data without prior training).

## How does KMeans work?

The way K-means clustering works is as follows:
- We start with a dataset of items, each with certain features and corresponding values.
- First, we randomly initialize **k** points (known as means or cluster centroids).
- Next, we assign each item to its closest mean and then update the mean's coordinates based on the averages of the items assigned to that cluster.
- This process is repeated for a specified number of iterations to determine the final clusters.

## Choosing the Number of Clusters
- The optimal number of clusters, **'k'**, for the algorithm is typically determined using methods like: **Elbow method** and  **Silhouette score**.

### Elbow Method
- The Elbow Method involves calculating the `within-cluster sum of squares (WCSS)` for a range of 'k' values and plotting these against **'k'**.
- **WCSS** is a measure of the total variance within each cluster. It quantifies how spread out the points within a cluster are from the cluster centroid.
- The within-cluster sum of squares (WCSS) is calculated as follows:

$$ \text{WCSS} = \sum_{i=1}^{k} \sum_{x \in C_i} \|x - \mu_i\|^2 $$
    
where:
- **k** is the number of clusters.
- **Ci** represents the i-th cluster.
- **x** is a data point in cluster **Ci**.
- **μi** is the centroid (mean) of cluster **Ci**.
- **|x - μi|^2** is the squared Euclidean distance between a data point **x** and its cluster centroid **μi**.

**Steps:**
1. Calculate WCSS for different values of **k**
    - Run the KMeans algorithm for a range of cluster numbers (e.g., k=1 to k=10).
    - For each k, compute the WCSS.
2. Plot **WCSS** against k:
    - Create a plot where the x-axis represents the number of clusters k and the y-axis represents the WCSS.
    - As k increases, the WCSS decreases because clusters become smaller and tighter.
3. Identify the **'elbow'** point:
    - The 'elbow' point on the plot is where the rate of decrease of WCSS slows down. This point indicates the optimal number of clusters.
    - Adding more clusters beyond this point does not significantly reduce the WCSS.
    - The "elbow" point, where the rate of decrease in WCSS slows down, is usually considered the optimal number of clusters.


### Silhouette score
- The silhouette score measures the quality of clustering by considering both the cohesion within clusters and the separation between clusters.
- The silhouette score `s` for a single data point `i` is calculated as follows:

$$s = (b - a)/max(a,b)$$

where:
- **a** represents the average distance from the data point to all other points in the same cluster
- **b** represents the minimum average distance from the data point to all points in the next nearest cluster
- The score ranges from -1 to 1, where a higher value indicates better-defined clusters.

**Steps:**
1. Silhouette scores for all points are calculated for a range of values for 'k'.
2. The value of 'k' with the highest average silhouette score is generally considered the optimal number of clusters.

From the scikit-learn library, the following site outlines the usage and parameters of the KMeans function.
https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#

**Objectives:**
- Simple K-Means Clustering Using Manual Data Points.
- K-mean on Synethetic data sample (using `make_blobs`) and Finding Relevant Statistics
- K-means Clustering on a Real-World Dataset.
- Practical Coding Example

## Install the necessary packages.

In [ ]:
%pip install numpy==1.26.0 pandas==2.2.2 scipy==1.10.1 jedi

In [ ]:
%pip install --upgrade --quiet scikit-learn matplotlib seaborn kneed

Authenticate your notebook environment (Colab only)
If you are running this notebook on Google Colab, run the cell below to authenticate your environment.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

Import the necessary libraries

In [ ]:
import tarfile
import urllib
import matplotlib.pyplot as plt
import numpy as np
from kneed import KneeLocator
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, adjusted_rand_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

import pandas as pd
import seaborn as sns

## Simple K-Means Clustering Using Manual Data Points

- First lets plot and visualize manual data points.

In [ ]:
# Visualize manual data points
x = [4, 5, 10, 4, 3, 11, 14, 6, 10, 12]
y = [21, 19, 24, 17, 16, 25, 24, 22, 21, 21]

plt.scatter(x, y)
plt.show()

- Next we calculate Inertia for Different Values of K, and visualize the result.

In [ ]:
data = list(zip(x, y))
inertias = []

for i in range(1, 11):
    kmeans = KMeans(n_clusters=i)
    kmeans.fit(data)
    inertias.append(kmeans.inertia_)

# Now we utilize the elbow method (discussed more later) to visualize the inertia for different values of K
plt.plot(range(1, 11), inertias, marker="o")
plt.xlabel("Number of clusters")
plt.ylabel("Inertia")
plt.show()

- From previous plot, we see that 2 is a good value for k, so we retrain and visualize clusters.

In [ ]:
kmeans = KMeans(n_clusters=2)
kmeans.fit(data)

plt.scatter(x, y, c=kmeans.labels_)
plt.show()

## K-means on Synthetic data sample (using `make_blobs`) and Finding Relevant Statistics

- Using make_blobs for Synthetic Data

In [ ]:
features, true_labels = make_blobs(
    n_samples=500, n_features=2, centers=3, random_state=23
)

fig = plt.figure(0)
plt.grid(True)
plt.scatter(features[:, 0], features[:, 1])
plt.show()

- **NOTE:** the `random_state` parameter is here to set a reproducible output
- In practice, leave `random_state` out, with its default value of `None`
- Let's take a look at the first five elements of each of the variables returned by `make_blobs()`

In [ ]:
features[:5]
true_labels[:5]

- Now, let scale the values  for each feature in the dataset.

In [ ]:
# Scale the values for each feature in the dataset
scalar = StandardScaler()
scaled_features = scalar.fit_transform(features)
scaled_features[:5]

- Instantiate a sample K-Means class

In [ ]:
# Instantiate a sample K-Means class
kmeans = KMeans(init="random", n_clusters=3, n_init=10, max_iter=300, random_state=42)

- Fit the k-means class to the data in scaled_features, and analyze their parameters.

In [ ]:
# Fit the k-means class to the data in scaled_features
kmeans.fit(scaled_features)


# The lowest SSE value
kmeans.inertia_

# Final locations of the centroid
kmeans.cluster_centers_

# The number of iterations required to converge
kmeans.n_iter_

# The first five predicted labels
kmeans.labels_[:5]

## Implementation of K-Means Clustering

- **Initialization**: Initialize the cluster centers randomly and create an empty list of points for each cluster.
- **E-step (Expectation)**: Assign each data point to the nearest cluster center.
- **M-step (Maximization)**: Recalculate the cluster centers as the mean of the points assigned to each cluster.
- **Repeat**: Iterate the E-step and M-step until the cluster centers converge.

In [ ]:
k = 3
clusters = {}
np.random.seed(23)

for idx in range(k):
    center = 2 * (2 * np.random.random((features.shape[1],)) - 1)
    points = []
    cluster = {"center": center, "points": []}
    clusters[idx] = cluster

clusters

In [ ]:
# Plot the blobs with the initialized centers
plt.scatter(features[:, 0], features[:, 1])
plt.grid(True)
for i in clusters:
    center = clusters[i]["center"]
    plt.scatter(center[0], center[1], marker="*", c="red")
plt.show()

- **Euclidean distance** is a measure of the "straight line" distance between two points.
- In KMeans, it is used to measure the similarity (or dissimilarity) between two points in a multi-dimensional space.

In [ ]:
# Define Euclidean distance
def distance(p1, p2):
    return np.sqrt(np.sum((p1 - p2) ** 2))

- The **E-step** involves assigning each data point to the nearest cluster center using `assign_clusters` function.
- The **M-step** involves updating the cluster centers based on the points assigned to each cluster using `update_clusters` function.
- These steps are repeated iteratively until the cluster centers converge (i.e., they no longer change significantly between iterations).

In [ ]:
# Implementing E-step
def assign_clusters(features, clusters):
    for idx in range(features.shape[0]):
        dist = []

        curr_x = features[idx]

        for i in range(k):
            dis = distance(curr_x, clusters[i]["center"])
            dist.append(dis)
        curr_cluster = np.argmin(dist)
        clusters[curr_cluster]["points"].append(curr_x)
    return clusters


# Implementing the M-Step
def update_clusters(features, clusters):
    for i in range(k):
        points = np.array(clusters[i]["points"])
        if points.shape[0] > 0:
            new_center = points.mean(axis=0)
            clusters[i]["center"] = new_center

            clusters[i]["points"] = []
    return clusters


# Predict cluster for datapoints
def pred_cluster(features, clusters):
    pred = []
    for i in range(features.shape[0]):
        dist = []
        for j in range(k):
            dist.append(distance(features[i], clusters[j]["center"]))
        pred.append(np.argmin(dist))
    return pred


# Assign, update, and predict the cluster center
clusters = assign_clusters(features, clusters)
clusters = update_clusters(features, clusters)
pred = pred_cluster(features, clusters)

In [ ]:
# Finally, plot the data points with their predicted cluster center
plt.scatter(features[:, 0], features[:, 1], c=pred)
for i in clusters:
    center = clusters[i]["center"]
    plt.scatter(center[0], center[1], marker="^", c="red")
plt.show()

## Choosing the Number of Clusters

- First lets find the optimal number of clusters (k) for K-means clustering using the **Elbow Method**.

In [ ]:
kmeans_kwargs = {"init": "random", "n_init": 10, "max_iter": 300, "random_state": 42}

# save the SSE values for each k value
sse = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(scaled_features)
    sse.append(kmeans.inertia_)

In [ ]:
# plot the number of clusters versus the SSE value to find/visualize the "elbow point"
plt.style.use("fivethirtyeight")
plt.plot(range(1, 11), sse)
plt.xticks(range(1, 11))
plt.xlabel("Number of Clusters")
plt.ylabel("SSE")
plt.show()

- The `KneeLocator` from the `kneed` library can be used to find the "**elbow**" point programmatically

In [ ]:
# Use KneeLocator to identify the elbow point programmatically if needed
kl = KneeLocator(range(1, 11), sse, curve="convex", direction="decreasing")

kl.elbow

- Next, lets find the optimal number of clusters (k) for K-means clustering using the **Silhouette Score**.

In [ ]:
# Find the silhouette coefficients and save them in a list for each k value
silhouette_coefficients = []

for k in range(2, 11):
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(scaled_features)
    score = silhouette_score(scaled_features, kmeans.labels_)
    silhouette_coefficients.append(score)

In [ ]:
# Similarly plot the number of clusters versus the silhouette coefficient to find the optimal k value
plt.style.use("fivethirtyeight")
plt.plot(range(2, 11), silhouette_coefficients)
plt.xticks(range(2, 11))
plt.xlabel("Number of Clusters")
plt.ylabel("Silhouette Coefficient")
plt.show()

## K-means Clustering on a Real-World Dataset

- Lets experiment with k-means clustering on a real-world dataset.
- We will be using the TCGA (The Cancer Genome Atlas) database which catalogs genomic alterations responsible for cancer.

- Import the necessary modules for extracting data and building/visualizing the data

In [ ]:
uci_tcga_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00401/"
archive_name = "TCGA-PANCAN-HiSeq-801x20531.tar.gz"

# Build the url
full_download_url = urllib.parse.urljoin(uci_tcga_url, archive_name)

# Download the file
r = urllib.request.urlretrieve(full_download_url, archive_name)

# Extract the data from the archive
tar = tarfile.open(archive_name, "r:gz")
tar.extractall()
tar.close()

- Load the **TCGA** data from file

In [ ]:
# load the data from the text file as NumPy arrays
datafile = "TCGA-PANCAN-HiSeq-801x20531/data.csv"
labels_file = "TCGA-PANCAN-HiSeq-801x20531/labels.csv"

data = np.genfromtxt(datafile, delimiter=",", usecols=range(1, 20532), skip_header=1)

true_label_names = np.genfromtxt(
    labels_file, delimiter=",", usecols=(1,), skip_header=1, dtype="str"
)

In [ ]:
# check the data and labels for the first five samples
data[:5, :3]
true_label_names[:5]

- Convert the labels to integers to allow for usage in evaluation.

In [ ]:
# convert the labels to integers to allow for usage in evaluation
label_encoder = LabelEncoder()

true_labels = label_encoder.fit_transform(true_label_names)
true_labels[:5]

# store the length of the array of classes as the number of clusters
label_encoder.classes_

n_clusters = len(label_encoder.classes_)

- Build a preprocessing pipeline and use MinMaxScaler for feature scaling.
- Implement the PCA class to perform dimensionality reduction.

In [ ]:
# build a preprocessing pipeline and use MinMaxScaler for feature scaling, implement the PCA class to perform dimensionality reduction
preprocessor = Pipeline(
    [
        ("scaler", MinMaxScaler()),
        ("pca", PCA(n_components=2, random_state=42)),
    ]
)

- Build another pipeline to perform k-means clustering.

In [ ]:
# Now build another pipeline to perform k-means clustering
clusterer = Pipeline(
    [
        (
            "kmeans",
            KMeans(
                n_clusters=n_clusters,
                init="k-means++",
                n_init=50,
                max_iter=500,
                random_state=42,
            ),
        ),
    ]
)

- Combine both the pipelines and train on the loaded dataset

In [ ]:
# combine the two pipelines into one
pipe = Pipeline([("preprocessor", preprocessor), ("clusterer", clusterer)])

# use .fit() to perform all the pipeline steps on the data
pipe.fit(data)

-  Evaluate performance by calculating the **Silhouette Score**.

In [ ]:
# calculate the silhouette score to evaluate performance
preprocessed_data = pipe["preprocessor"].transform(data)

predicted_labels = pipe["clusterer"]["kmeans"].labels_

silhouette_score(preprocessed_data, predicted_labels)

# calculate the adjusted rand index (ARI)
adjusted_rand_score(true_labels, predicted_labels)

- Visualise the results using graphs.

In [ ]:
# now visualize the data by plotting the results
pcadf = pd.DataFrame(
    pipe["preprocessor"].transform(data),
    columns=["component_1", "component_2"],
)

pcadf["predicted_cluster"] = pipe["clusterer"]["kmeans"].labels_
pcadf["true_label"] = label_encoder.inverse_transform(true_labels)

plt.style.use("fivethirtyeight")
plt.figure(figsize=(8, 8))

scatter = sns.scatterplot(
    x="component_1",
    y="component_2",
    s=50,
    data=pcadf,
    hue="predicted_cluster",
    style="true_label",
    palette="Set2",
)

scatter.set_title("Clustering results from TCGA Pan-Cancer\nGene Expression Data")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)
plt.show()

### Practical Coding Example:

- In this section, first use `make_blobs` to create synthetic data.
- Then perform k-means clustering and plot the data.
- Then load the **iris** dataset from scikit-learn.
- Perform k-means clustering on the dataset, and then plot the clusters.

In [ ]:
# Import the necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs, load_iris
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
# Create synthetic data using make_blobs, fill in make_blobs
n_samples = 300
n_features = 2
n_clusters = 3
X, y = make_blobs()

In [ ]:
# Plot the synthetic data


In [ ]:
# Find and plot the optimal number of clusters using the elbow method
inertia = []
K = range(1, 11)
for k in K:


In [ ]:
# Perform KMeans clustering on the synthetic data. using the k value determined visually from the previous plot


In [ ]:
# Plot the clusters


In [ ]:
# load the dataset load_iris, which will be used for this example
iris = load_iris()
X_iris = iris.data
y_iris = iris.target

In [ ]:
# Standardize the Iris dataset
scaler = StandardScaler()
X_iris_scaled = scaler.fit_transform(X_iris)

In [ ]:
# Find and plot the optimal number of clusters using the elbow method
inertia_iris = []
K_iris = range(1, 11)
for k in K_iris:

In [ ]:
# Perform KMeans clustering on the Iris dataset


In [ ]:
# Reduce dimensions for visualization using PCA


In [ ]:
# Plot the Iris dataset clusters


### Important information to remember

1. What is the overall objective of k-means clustering?
    - The objective of K-Means clustering is to partition a dataset into k distinct, non-overlapping subsets, or 'clusters,' where the data points within each cluster are similar to each other and different from those in other clusters.
    - In a more technical sense, K-Means aims to minimize the within-cluster variance, which is typically measured using squared Euclidean distances.

2. What are some situations where using K-Means is the best choice, and what are some where it isn't the best choice?
    1. Some situations where k-means is useful include:
        - data points form well-separated, spherical clusters
        - dataset has relatively low number of features, where Euclidean distance is more meaningful
        - dataset is large and/or clusters are expected to be similar sized
        - speed is needed, as k-means is a very fast clustering algorithm

    2. Some situations where k-means is sub-optimal include:
        - clusters are irregularly shaped, as k-means assumes clusters are convex and spherical
        - clusters are different sizes, as k-means generally produces clusters of similar sizes
        - there are outliers, as k-means is sensitive to outliers, skewing the centroid positions
        - dataset has many dimensions/features, as the distance measures used by k-means can be less meaningful

    3. Some alternative algorithms for sub-optimal situations:
        - DBSCAN for non-convex clusters and noisy data
        - Gaussian Mixture Models for clusters of different sizes and shapes
        - Other algorithms that best fit the specific characteristics of the dataset

The primary goals of this notebook were to grasp the K-Means clustering algorithm and its functioning. With this knowledge, you should be able to implement your own K-Means clustering code and visualize the K-Means process. Hopefully, this notebook has achieved that and has enhanced your understanding of K-Means.

### Resources
- https://towardsdatascience.com/k-means-clustering-algorithm-applications-evaluation-methods-and-drawbacks-aa03e644b48a
- https://scikit-learn.org/1.5/modules/generated/sklearn.cluster.KMeans.html